In [1]:
import os
import joblib
from utils import pretty_trim, simple_split
from collections import Counter
from scipy.sparse import vstack
import numpy as np
import chardet

** Loading trained models **

In [2]:
%%time
filename = 'models_persistence/pickle_models'
(pretty_trim, counter, tfidf, rfe, clfs) = joblib.load(filename)

Wall time: 21.1 s


In [3]:
%%time
filename = 'models_persistence/final_dataset'
(X_train_final, y_train, X_test_final, y_test) = joblib.load(filename)

Wall time: 1.6 s


In [4]:
str_labels = [u'0 บริหารธุรกิจ',
    u'1 ประมง',
    u'2 มนุษยศาสตร์',
    u'3 วนศาสตร์',
    u'4 วิทยาการจัดการ',
    u'5 วิทยาศาสตร์',
    u'6 วิทยาศาสตร์การกีฬา',
    u'7 วิศวกรรมศาสตร์',
    u'8 ศิลปศาสตร์และวิทยาศาสตร์',
    u'9 ศึกษาศาสตร์',
    u'10 ศึกษาศาสตร์และพัฒนศาสตร์',
    u'11 สถาปัตยกรรมศาสตร์',
    u'12 สังคมศาสตร์',
    u'13 สัตวแพทยศาสตร์',
    u'14 สิ่งแวดล้อม',
    u'15 อุตสาหกรรมเกษตร',
    u'16 เกษตร',
    u'17 เศรษฐศาสตร์',
    u'18 โครงการจัดตั้งวิทยาเขตสุพรรณบุรี',
    u'19 โครงการสหวิทยาการระดับบัณฑิตศึกษา']

In [62]:
# # refitting on the whole dataset
# X_old, y_old = vstack([X_train_final, X_test_final]), np.concatenate([y_train, y_test])
# clf.fit(X_old, y_old)

In [5]:
clf = clfs[0]
print 'score:', clf.score(X_test_final, y_test)
proba = clf.predict_proba(X_test_final)
confidence = proba.max(1)
print 'confidence:', confidence.mean(), confidence.std(), confidence.min(), confidence.max()

score: 0.927450980392
confidence: 0.73368627451 0.220944806363 0.16 1.0


** Read segmented docs **

In [6]:
%%time
doc_path = u'./corpus/segmented-journal' # must be a segmented doc path
dataset_contents = []
filename2index = dict()
for i, filename in enumerate(os.listdir(doc_path)):
    path = os.path.join(doc_path, filename)
    filename2index[filename] = i
    with open(path) as f:
        content = f.read()
#         if chardet.detect(content)['encoding'] == 'ascii':
#             continue
        content = content.decode('utf8')
        dataset_contents.append(content)
print 'total files:', len(dataset_contents)

total files: 2165
Wall time: 29.8 s


# Apply learning pipeline to all the docs
First trim

In [22]:
%%time
for i in xrange(len(dataset_contents)):
    dataset_contents[i] = pretty_trim(dataset_contents[i])

Wall time: 5.35 s


Then count words and apply Tf-idf

In [117]:
%time X_new_count = counter.transform(dataset_contents)
%time X_new_tfidf = tfidf.transform(X_new_count)
print X_new_tfidf.shape

Wall time: 14.5 s
Wall time: 164 ms
(2165, 250000)


Remove some features

In [118]:
%time X_new_rfe = rfe.transform(X_new_tfidf)
print X_new_rfe.shape

Wall time: 94 ms
(2165, 20000)


Predict using trained models

In [119]:
y_pred = clf.predict(X_new_rfe)
Counter(y_pred)

Counter({0: 1,
         2: 1,
         3: 1,
         5: 1322,
         7: 15,
         9: 32,
         12: 2,
         15: 192,
         16: 575,
         17: 17,
         19: 7})

In [120]:
proba = clf.predict_proba(X_new_rfe)
confidence = proba.max(1)
print 'confidence:', confidence.mean(), confidence.std(), confidence.min(), confidence.max()

confidence: 0.457131639723 0.130549181945 0.13 0.92


Evaluate predict_proba on some files to roughly interpret the performance

In [121]:
filenames = ['A0906251543307343.txt', 'A1410221625383281.txt', 'A1006241011134591.txt', 'A1004071348071718.txt']
# these are files that contain word 'department'
indices = [filename2index[filename] for filename in filenames]
labels = [str_labels[pred] for pred in y_pred[indices]]
for label, prob in zip(labels, confidence[indices]):
    print label, prob

5 วิทยาศาสตร์ 0.33
5 วิทยาศาสตร์ 0.63
5 วิทยาศาสตร์ 0.52
5 วิทยาศาสตร์ 0.51


In [122]:
# what is the label of all docs that contain word 'rice' ?
preds = []
for i, content in enumerate(dataset_contents):
    if u'rice' in content.split():
        preds.append((str_labels[y_pred[i]], confidence[i]))
preds.sort(key=lambda item: item[1], reverse=True)
for pred in preds:
    print pred[0], pred[1]

5 วิทยาศาสตร์ 0.78
5 วิทยาศาสตร์ 0.77
5 วิทยาศาสตร์ 0.76
5 วิทยาศาสตร์ 0.76
5 วิทยาศาสตร์ 0.75
5 วิทยาศาสตร์ 0.75
5 วิทยาศาสตร์ 0.74
5 วิทยาศาสตร์ 0.74
5 วิทยาศาสตร์ 0.73
5 วิทยาศาสตร์ 0.73
5 วิทยาศาสตร์ 0.73
5 วิทยาศาสตร์ 0.72
5 วิทยาศาสตร์ 0.72
5 วิทยาศาสตร์ 0.71
5 วิทยาศาสตร์ 0.7
5 วิทยาศาสตร์ 0.7
16 เกษตร 0.7
5 วิทยาศาสตร์ 0.69
5 วิทยาศาสตร์ 0.69
5 วิทยาศาสตร์ 0.68
5 วิทยาศาสตร์ 0.68
5 วิทยาศาสตร์ 0.68
16 เกษตร 0.68
5 วิทยาศาสตร์ 0.68
5 วิทยาศาสตร์ 0.67
16 เกษตร 0.67
5 วิทยาศาสตร์ 0.67
5 วิทยาศาสตร์ 0.67
5 วิทยาศาสตร์ 0.67
5 วิทยาศาสตร์ 0.65
5 วิทยาศาสตร์ 0.65
16 เกษตร 0.65
5 วิทยาศาสตร์ 0.64
5 วิทยาศาสตร์ 0.64
5 วิทยาศาสตร์ 0.64
16 เกษตร 0.63
5 วิทยาศาสตร์ 0.63
5 วิทยาศาสตร์ 0.63
5 วิทยาศาสตร์ 0.63
16 เกษตร 0.63
16 เกษตร 0.63
5 วิทยาศาสตร์ 0.62
5 วิทยาศาสตร์ 0.62
16 เกษตร 0.62
16 เกษตร 0.62
5 วิทยาศาสตร์ 0.61
16 เกษตร 0.61
16 เกษตร 0.61
5 วิทยาศาสตร์ 0.61
5 วิทยาศาสตร์ 0.61
15 อุตสาหกรรมเกษตร 0.61
5 วิทยาศาสตร์ 0.61
15 อุตสาหกรรมเกษตร 0.6
5 วิทยาศาสตร์ 0.6
5 วิทยาศาสตร์ 0.59
16 เก

## Synthesize approximated labels using heuristic

In [224]:
# word to class label mapping dictioanry
approx_label = {
#     "liber": 8,
#     "art": 8,
    "agricultur": 16,
    "agro": 16,
    "educ": 9,
#     "social": 12,
#     "fisheri": 1,
#     "manag": 4,
    "scienc": 5,
    "technolog": 5,
#     "medicin": 13,
#     "pharmaci": 13,
#     "forestri": 3,
#     "forest": 3,
    "engin": 7,
    "econom": 17,
    "architectur": 11,
#     "human": 2,
    "biotechnolog": 5,
#     "environment": 14,
#     "environ": 14,
#     "veterinari": 13,
#     "busi": 0,
#     u"ธุรกิจ": 0,
}

Find approximated labels by searching for the faculty name

In [225]:
heuristic_y = np.zeros(len(dataset_contents), dtype=np.int32) - 1 # starts with -1 filled
for ci in range(len(dataset_contents)):
    words = dataset_contents[ci].split()
    contexts = []
    wis = []
    for wi, word in enumerate(words):
        if u'faculti' in word or u'คณะ' in word:
            context = words[wi-3:wi+5]
            contexts.append(context)
            wis.append(wi)
            for w in context:
                if w in approx_label:
                    heuristic_y[ci] = approx_label[w]
                    break
        if heuristic_y[ci] != -1:
            break
    if contexts: # logging
        label = str_labels[heuristic_y[ci]] if heuristic_y[ci] != -1 else 'UNKNOWN'
        print 'Document No.', ci, '(', label, ')'
        
        for i in range(len(contexts)):
            print 'Word No.', wis[i], ' => ',
            for w in contexts[i]:
                if w in approx_label:
                    w = '[%s]' % w
                print w,
            print

Document No. 0 ( 16 เกษตร )
Word No. 291  =>  of plant patholog faculti of [agricultur] kasetsart univers
Document No. 1 ( 5 วิทยาศาสตร์ )
Word No. 12  =>  2550 [scienc] [educ] faculti of [educ] kasetsart univers
Document No. 2 ( 5 วิทยาศาสตร์ )
Word No. 23  =>  for [scienc] [educ] faculti of [educ] kasetsart univers
Document No. 3 ( 17 เศรษฐศาสตร์ )
Word No. 15  =>  home [econom] [educ] faculti of [educ] kasetsart univers
Document No. 4 ( UNKNOWN )
Word No. 22  =>  univers chiangrai thailand faculti of medicin maha sara
Document No. 5 ( 9 ศึกษาศาสตร์ )
Word No. 15  =>  depart of [educ] faculti of [educ] kasetsart univers
Document No. 7 ( 16 เกษตร )
Word No. 18  =>  extens and communic faculti of [agricultur] kasetsart univers
Document No. 8 ( 16 เกษตร )
Word No. 18  =>  extens and communic faculti of [agricultur] at kamphaeng
Document No. 10 ( 16 เกษตร )
Word No. 18  =>  extens and communic faculti of [agricultur] at kamphaeng
Document No. 11 ( 5 วิทยาศาสตร์ )
Word No. 23  =>  muang p

ValueError: I/O operation on closed file

In [226]:
print 'Total Label Approximations:', np.count_nonzero(heuristic_y != -1)

Total Label Approximations: 1162


### Evaluate accuracy score on approximated labels

In [227]:
from sklearn.metrics import accuracy_score

# test data that do not have approximated label will not be used
valid_mask = heuristic_y != -1
print 'Accuracy:', accuracy_score(heuristic_y[valid_mask], y_pred[valid_mask])

# count labels distribution
Counter(heuristic_y[valid_mask]), Counter(y_pred[valid_mask])

Accuracy: 0.581755593804


(Counter({5: 648, 7: 86, 9: 48, 11: 4, 16: 344, 17: 32}),
 Counter({5: 740, 7: 7, 9: 4, 15: 125, 16: 280, 17: 2, 19: 4}))